# 11_接口：从协议到抽象基类

## 11.1 序列协议

Sequence
- `__getitem__`
- `__contains__`
- `__iter__`
- `__reversed__`
- `__len__`
- `count`

In [3]:
# 支持洗牌的类
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) 
                       for suit in self.suits 
                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def inset(self, position, value):
        self._cards.inset(position, value)

In [4]:
dir(FrenchDeck2)

['__abstractmethods__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'append',
 'clear',
 'count',
 'extend',
 'index',
 'insert',
 'inset',
 'pop',
 'ranks',
 'remove',
 'reverse',
 'suits']

## 11.3 标准库中的抽象基类

### 11.3.1 `collections.abc` 模块中的抽象基类

- `Iterable / Container / Sized`
    * `__iter__`，支持迭代；
    * `__contains__` 支持 `in` 运算符；
    * `__len__` 支持 `len()` 方法。
- `Sequence / Mapping / Set`
- `MappingView`
    * 支持映射方法 `.items(), .keys(), .values()` 
- `Callable / Hashable`
- `Iterator`
    * `Iterator` 的子类；

### 11.3.2 `numbers`

#### 1 基类的内容：
- `Number`
- `Complex`
- `Real`
- `Rational`
- `Integral`

#### 2 应用：
- 检查是否是整数 `isinstance(x, numbers.Integral)`
- 检查是否是浮点数 `isinstance(x, numbers.Real)`

## 11.7 定义并使用一个抽象基类

In [2]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod   # abstractmethod() 应该放在最里层
    def load(self, iterable):
        """ 从可迭代对象中添加元素 """
        
    @abc.abstractmethod
    def pick(self):
        """ 随机删除元素，然后将其返回。 
        如果实力为空，这个方法应该抛出 LookupError。
        """
    
    def loaded(self):
        """ 如果至少有一个元素，返回True，否则返回False。 """
        return bool(self.inspect())
    
    def inspect(self):
        """ 返回一个有序元组，由当前元素构成。 """
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [15]:
class Fake(Tombola):
    def pick(self):
        return 13
    
print(Fake)

try:
    f = Fake()
except Exception as e:
    print(f"Error: {e}")

<class '__main__.Fake'>
Error: Can't instantiate abstract class Fake with abstract methods load


In [17]:
import random

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [21]:
b = BingoCage([1,2,3])
b